In [140]:
import Modules.SQcircuit_extensions as sq_ext
import Modules.figures as figs
import SQcircuit as sq
import numpy as np
import matplotlib.pyplot as plt
import importlib
from scipy.linalg import eigh
import qutip as qt
import sympy as sym
from joblib import Parallel, delayed, cpu_count

%matplotlib ipympl
# plt.rcParams.update({
#     "text.usetex": True,
#     "font.family": "serif",
#     "font.serif": ["Palatino"],
# })
plt.rcParams['text.usetex'] = False
importlib.reload(sq_ext)
importlib.reload(sq)
importlib.reload(figs)
np.set_printoptions(linewidth=300, formatter={'float': '{:.3e'.format})

In [141]:
n_eig =5

In [142]:
fF  = 1e-15
nH  = 1e-9
GHz = 1e9

In [143]:
nmax_r = 20
nmax_f = 25

only_inner = True
compensate_extra_cap = False

In [144]:
CR_1, CF_1, LF_1, LR_1, EJ_1, Δ_1, ω_r_1 = sq_ext.get_experimental_parameters('qubit_1')
CR_2, CF_2, LF_2, LR_2, EJ_2, Δ_2, ω_r_2 = sq_ext.get_experimental_parameters('qubit_2')
CR_3, CF_3, LF_3, LR_3, EJ_3, Δ_3, ω_r_3 = sq_ext.get_experimental_parameters('qubit_3')

## Single qubit

In [196]:
## Testings
Δ=0.1
fluxonium = sq_ext.sq_fluxonium(C_F_eff=CF_1, L_F_eff=LF_1, EJ=EJ_1, Δ=Δ_1, nmax_f=nmax_f)
resonator = sq_ext.sq_resonator(C_R_eff=CR_1, L_R_eff=LR_1, Δ=Δ_1, nmax_f=nmax_f)

H_0 = sq_ext.hamiltonian_qubit(fluxonium = fluxonium, resonator = resonator, Δ=0) 
H   = sq_ext.hamiltonian_qubit(fluxonium = fluxonium, resonator = resonator, Δ=Δ) 

H1 = fluxonium.hamiltonian()
H2 = resonator.hamiltonian()

ψ_0_f  = qt.Qobj(sq_ext.diag(H1, n_eig=2,solver='numpy')[1][:,0])
ψ_1_f  = qt.Qobj(sq_ext.diag(H1, n_eig=2,solver='numpy')[1][:,1])
ψ_0_r  = qt.Qobj(sq_ext.diag(H2, n_eig=2,solver='numpy')[1][:,0])
ψ_1_r  = qt.Qobj(sq_ext.diag(H2, n_eig=2,solver='numpy')[1][:,1])

ψ_0 = [qt.tensor(ψ_0_f, ψ_0_r), qt.tensor(ψ_1_f,ψ_0_r), qt.tensor(ψ_0_f,ψ_1_r), qt.tensor(ψ_1_f,ψ_1_r)]

In [197]:
H_eff_SWT = sq_ext.H_eff_SWT(H_0, H, n_eig=5)
Nf, Nr = sq_ext.sq_get_energy_indices_hamiltonian(H_qubit = H_0, H_fluxonium=H1, H_resonator=H2, n_eig=5)
subspace_indices = np.intersect1d(np.where(Nf<=1), np.where(Nr<=1)).tolist()

H_eff_SWT_subspace  = H_eff_SWT [subspace_indices][:, subspace_indices]
P = sq_ext.decomposition_in_pauli_4x4(H_eff_SWT_subspace, print_pretty=True)

 9.885452537414066	*	 I ⨂I 
 -1.8442461348076464	*	 I ⨂σz 
 2.192745890694514e-07	*	 σx ⨂σx 
 0.020418719823306453	*	 σy ⨂σy 
 -3.229888141708264	*	 σz ⨂I 
 -7.072425806819993e-05	*	 σz ⨂σz 


In [193]:
E_0  = [ (ψ_0_i.dag() * H_0 * ψ_0_i).data[0,0]/2/np.pi/GHz for ψ_0_i in ψ_0]
E, ψ = sq_ext.diag(H  , n_eig, out='GHz', solver='Qutip', qObj=True)
subspace_indices = sq_ext.find_closest_indices(E_0,E,tolerance=0.05)
ψ = ψ[subspace_indices]
E = E[subspace_indices]
H_eff_SWT_large = sq_ext.H_eff_SWT_large(ψ_0, ψ, E)
P = sq_ext.decomposition_in_pauli_4x4(H_eff_SWT_large, print_pretty=True)

 9.8855373788609	*	 I ⨂I 
 -1.8443309762858033	*	 I ⨂σz 
 -0.020418928565167773	*	 σx ⨂σx 
 -1.0532714364748508e-08	*	 σy ⨂σy 
 -3.2299729831865003	*	 σz ⨂I 
 1.411718873023382e-05	*	 σz ⨂σz 
